# Merging datasets

In [1]:
# setup
import pandas as pd
import numpy as np

df1 = pd.DataFrame(np.arange(16).reshape(4,4), columns=list("ABCD"))
df2 = pd.DataFrame(np.zeros((3,4)), columns=list("ABCD"))
df1

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [2]:
np.zeros([3,4])

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [3]:
df2

,A,B,C,D
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


## Concat
- concetenates along a particular axis
- set logic

In [4]:
pd.concat([df1, df2])
# df1 and df2 have the same columns name
# index are same as other file

,A,B,C,D
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0
3,12.0,13.0,14.0,15.0
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


In [5]:
pd.concat([df1, df2]).reset_index(drop=True)

,A,B,C,D
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0
3,12.0,13.0,14.0,15.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0


In [6]:
pd.concat([df1, df2], axis=1)
# df1 shape (4,4), df2 shape (3,4)

,A,B,C,D,A,B,C,D
0,0,1,2,3,0.0,0.0,0.0,0.0
1,4,5,6,7,0.0,0.0,0.0,0.0
2,8,9,10,11,0.0,0.0,0.0,0.0
3,12,13,14,15,NaN,NaN,NaN,NaN


In [7]:
pd.concat([df1, df2], axis=1, join="inner")

,A,B,C,D,A,B,C,D
0,0,1,2,3,0.0,0.0,0.0,0.0
1,4,5,6,7,0.0,0.0,0.0,0.0
2,8,9,10,11,0.0,0.0,0.0,0.0


## Merge

- high performance join operations similar to relational databases
- performance is faster using merge/join in Pandas than in SQL

Relational algebra
- one-to-one - joining 2 dfs on their index
- many-to-one - joining a unique index to ≥ 1 cols in different df
- many-to-many - joining columns on columns

<img align = "left" src="../assets/set_logic.png"> 

In [8]:
# merge
left = pd.DataFrame({"key": ["K0", "K0", "K1", "K2"],
                     "A": ["A0", "A1", "A2", "A3"],
                     "B": ["B0", "B1", "B2", "B3"],
                     })
right = pd.DataFrame({"key": ["K0", "K1", "K2", "K3"],
                     "C": ["C0", "C1", "C2", "C3"],
                      "D": ["D0", "D1", "D2", "D3"]})
print(left)
print(right)

  key   A   B
0  K0  A0  B0
1  K0  A1  B1
2  K1  A2  B2
3  K2  A3  B3
  key   C   D
0  K0  C0  D0
1  K1  C1  D1
2  K2  C2  D2
3  K3  C3  D3


In [9]:
pd.merge(left, right, on="key", indicator=True) # how= "inner" default

,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both


In [10]:
left.merge(right, on=["key"], how="outer", indicator=True)

,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both
4,K3,NaN,NaN,C3,D3,right_only


In [11]:
left.merge(right, on="key", how="left", indicator=True)

,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both


In [12]:
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "A": ["A0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

print(left)

print(right)

# only keys present in left
left.merge(right, on=["key1", "key2"], how="left", indicator=True)

  key1 key2   A   B
0   K0   K0  A0  B0
1   K0   K1  A1  B1
2   K1   K0  A2  B2
3   K2   K1  A3  B3
  key1 key2   A   D
0   K0   K0  A0  D0
1   K1   K0  C1  D1
2   K1   K0  C2  D2
3   K2   K0  C3  D3


,key1,key2,A_x,B,A_y,D,_merge
0,K0,K0,A0,B0,A0,D0,both
1,K0,K1,A1,B1,NaN,NaN,left_only
2,K1,K0,A2,B2,C1,D1,both
3,K1,K0,A2,B2,C2,D2,both
4,K2,K1,A3,B3,NaN,NaN,left_only


In [13]:
# keys in right
left.merge(right, on=["key1", "key2"], how="right", indicator=True, suffixes=["_left","_right"])

,key1,key2,A_left,B,A_right,D,_merge
0,K0,K0,A0,B0,A0,D0,both
1,K1,K0,A2,B2,C1,D1,both
2,K1,K0,A2,B2,C2,D2,both
3,K2,K0,NaN,NaN,C3,D3,right_only


## Join
- combine cols of two potentially different index dfs

In [14]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)


right = pd.DataFrame(
    {"C": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)

left.join(right) # joins on left
# ttps://datascienceparichay.com/article/pandas-join-vs-merge/#:~:text=Difference%20between%20pandas%20join%20and%20merge%20Both%20the,join%20dataframes%20on%20indexes%20as%20well%20as%20columns
# Difference between pandas join and merge
# Both the functions are used to perform joins on pandas dataframes but they’re used in different scenarios. The join() function is generally used to join dataframes on index whereas the merge() function is a more versatile function that lets you join dataframes on indexes as well as columns.


,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


## Read_html

In [15]:
link = "https://en.wikipedia.org/wiki/List_of_potentially_habitable_exoplanets"
tables = pd.read_html(link)
tables

[                Object              Star Star type  Mass (M⊕) Radius (R⊕)  \
 0                Earth         Sun (Sol)       G2V       1.00        1.00   
 1   Teegarden's Star b  Teegarden's Star       M7V      ≥1.05           —   
 2            TOI 700 d           TOI 700       M2V       1.72        1.14   
 3               K2-72e             K2-72       M?V      ~2.21        1.29   
 4          TRAPPIST-1d        TRAPPIST-1       M8V       0.30        0.78   
 ..                 ...               ...       ...        ...         ...   
 56        Gliese 357 d        Gliese 357       M2V      ≥6.10           —   
 57        Gliese 625 b        Gliese 625       M2V  2.82±0.51         NaN   
 58          Kepler-26e         Kepler-26         K        NaN         2.1   
 59         Kepler-737b        Kepler-737         M        4.5        1.96   
 60     Luyten 98-59 f*      Luyten 98-59       M3V       2.46         NaN   
 
    Density (g/cm3) Flux (F⊕) Teq (K)  Period (days)  Distance

In [16]:
df = tables[0] # first table in the html page 
df.head()

,Object,Star,Star type,Mass (M⊕),Radius (R⊕),Density (g/cm3),Flux (F⊕),Teq (K),Period (days),Distance (ly),Refs/Notes
0,Earth,Sun (Sol),G2V,1.00,1.00,5.514,1.00,255,365.25,0.00,[3]
1,Teegarden's Star b,Teegarden's Star,M7V,≥1.05,—,NaN,1.15,264,4.91,12.58,[4]
2,TOI 700 d,TOI 700,M2V,1.72,1.14,5.631,0.87,246,37.40,101.00,NaN
3,K2-72e,K2-72,M?V,~2.21,1.29,5.675,1.11,261,24.20,217.00,[5]
4,TRAPPIST-1d,TRAPPIST-1,M8V,0.30,0.78,3.39,1.04,258,4.05,39.00,Confirmed to be rocky[6][7]


In [17]:
fifa_tables = pd.read_html("https://en.wikipedia.org/wiki/FIFA_World_Cup", match="Highest attendances")

fifa_tables[0].head()

Year        Hosts Venues/Cities Totalattendance Matches Avg.attendance  \
   Year        Hosts Venues/Cities Totalattendance Matches Avg.attendance   
0  1930      Uruguay           3/1          590549      18          32808   
1  1934        Italy           8/8          363000      17          21353   
2  1938       France          10/9          375700      18          20872   
3  1950       Brazil           6/6         1045246      22          47511   
4  1954  Switzerland           6/6          768607      26          29562   

  Highest attendances †                                    \
                 Number                             Venue   
0                 93000    Estadio Centenario, Montevideo   
1                 55000        Stadio Nazionale PNF, Rome   
2                 58455      Olympique de Colombes, Paris   
3           173,850[82]  Maracanã Stadium, Rio de Janeiro   
4                 63000            Wankdorf Stadium, Bern   

                                       
                              Game(s)  
0  Uruguay 6–1 Yugoslavia, Semi-final  
1     Italy 2–1 Czechoslovakia, Final  
2     France 1–3 Italy, Quarter-final  
3  Brazil 1–2 Uruguay, Deciding match  
4     West Germany 3–2 Hungary, Final